In [1]:
$LOAD_PATH << File.dirname(__FILE__) + "/../lib"

require 'pandas'
require 'matplotlib/iruby'
Matplotlib::IRuby.activate
pd = Pandas
plt = Matplotlib::Pyplot

require 'beer'
include Beer

#<Module:0x000000010f7ec5b0>

# 異なる麦芽を混合した場合の数値(PPG単位)

In [2]:
# PPGの異なる麦芽を混合
mashing_recipe = MashingRecipe.new
  .add(500, Malt.ppg(40))
  .add(900, Malt.ppg(36))
  .add(300, Malt.ppg(35))
  .add(100, Malt.spec(dbcg: 0.82, mc: 0.04))

brewhouse_efficiency = 0.9
attenuation = 0.75

df = pd::DataFrame.new(mashing_recipe.malts.to_a, columns: ["Weight", "_malt"])
df.loc["Total"] = [mashing_recipe.weight, mashing_recipe.to_malt]

df["_ppg"] = df["_malt"].map(lambda{|malt| malt.to_ingredient(brewhouse_efficiency).to_ppg})
df["Brewhouse Yield"] = df["_malt"].map(lambda{|malt| malt.brewhouse_yield(brewhouse_efficiency)})
df["Plato"] = df["_malt"].map(lambda{|malt| malt.plato(brewhouse_efficiency)})
df["Specific Gravity"] = df["_malt"].map(lambda{|malt| malt.sg(brewhouse_efficiency)})
df["PPG"] = df["_malt"].map(lambda{|malt| malt.ppg(brewhouse_efficiency)})
df["ABV"] = df["_ppg"].map(lambda{|ppg| ppg.abv(attenuation)*100})

df = df.drop(df.columns.to_a.select{|s| s.start_with?("_")}, axis=1)
df

,Weight,Brewhouse Yield,Plato,Specific Gravity,PPG,ABV
0,500,0.778985,8.947419,1.036000,36.000000,3.658537
1,900,0.701086,8.052677,1.032400,32.400000,3.292683
2,300,0.681612,7.828991,1.031500,31.500000,3.201220
3,100,0.700200,8.042497,1.032359,32.359043,3.288521
Total,1800,0.719430,8.263370,1.033248,33.247725,3.378834


# 麦芽をマッシングした麦汁の数値

In [3]:
# PPGの異なる麦芽を混合してマッシング
wort = mashing_recipe.mash(10_000, brewhouse_efficiency)

data = {
  "Batch Size": wort.batch_size,
  "Brewhouse Efficiency": brewhouse_efficiency,
  "Brix": wort.to_ingredient.brix,
  "Original Gravity": wort.og,
  "ABV": wort.abv(attenuation) * 100,
}
pd::Series.new(data.values, index: data.keys)

|                      |           0 |
|:---------------------|------------:|
| Batch Size           | 10000       |
| Brewhouse Efficiency |     0.9     |
| Brix                 |    13.7809  |
| Original Gravity     |     1.0559  |
| ABV                  |     5.68112 |

# ホップを煮沸した麦汁の数値

In [4]:
boil_recipe = BoilRecipe.new
  .add(20, 60, Hop.new(6 * 0.01))

evaporation_rate = 0.05

wort
  .boil(boil_recipe, evaporation_rate)

data = {
  "Batch Size": wort.batch_size,
  "Brix": wort.to_ingredient.brix,
  "Attenuation": attenuation,
  "Original Gravity": wort.og,
  "Final Gravity": wort.fg(attenuation),
  "ABV": wort.abv(attenuation) * 100,
  "IBU": wort.ibu,
}
pd::Series.new(data.values, index: data.keys)

|                  |          0 |
|:-----------------|-----------:|
| Batch Size       | 9500       |
| Brix             |   14.4705  |
| Attenuation      |    0.75    |
| Original Gravity |    1.05884 |
| Final Gravity    |    1.01471 |
| ABV              |    5.98013 |
| IBU              |   33.923   |